In [4]:
import pandas as pd
import numpy as np
from scipy.stats import kurtosis
from pmdarima import auto_arima
import pmdarima as pm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping
from talib import abstract
import json


def mean_absolute_percentage_error(actual, prediction):
    actual = pd.Series(actual)
    prediction = pd.Series(prediction)
    return 100 * np.mean(np.abs((actual - prediction))/actual)


def get_arima(data, train_len, test_len):
    # prepare train and test data
    data = data.tail(test_len + train_len).reset_index(drop=True)
    train = data.head(train_len).values.tolist()
    test = data.tail(test_len).values.tolist()

    # Initialize model
    model = auto_arima(train, max_p=3, max_q=3, seasonal=False, trace=True,
                       error_action='ignore', suppress_warnings=True)

    # Determine model parameters
    model.fit(train)
    order = model.get_params()['order']
    print('ARIMA order:', order, '\n')

    # Genereate predictions
    prediction = []
    for i in range(len(test)):
        model = pm.ARIMA(order=order)
        model.fit(train)
        print('working on', i+1, 'of', test_len, '-- ' + str(int(100 * (i + 1) / test_len)) + '% complete')
        prediction.append(model.predict()[0])
        train.append(test[i])

    # Generate error data
    mse = mean_squared_error(test, prediction)
    rmse = mse ** 0.5
    mape = mean_absolute_percentage_error(pd.Series(test), pd.Series(prediction))
    return prediction, mse, rmse, mape


def get_lstm(data, train_len, test_len, lstm_len=4):
    # prepare train and test data
    data = data.tail(test_len + train_len).reset_index(drop=True)
    dataset = np.reshape(data.values, (len(data), 1))
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset_scaled = scaler.fit_transform(dataset)
    x_train = []
    y_train = []
    x_test = []

    for i in range(lstm_len, train_len):
        x_train.append(dataset_scaled[i - lstm_len:i, 0])
        y_train.append(dataset_scaled[i, 0])
    for i in range(train_len, len(dataset_scaled)):
        x_test.append(dataset_scaled[i - lstm_len:i, 0])

    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    # Set up & fit LSTM RNN
    model = Sequential()
    model.add(LSTM(units=lstm_len, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(LSTM(units=int(lstm_len/2)))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='mean_squared_error', optimizer='adam')
    early_stopping = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
    model.fit(x_train, y_train, epochs=500, batch_size=1, verbose=2, callbacks=[early_stopping])

    # Generate predictions
    prediction = model.predict(x_test)
    prediction = scaler.inverse_transform(prediction).tolist()

    output = []
    for i in range(len(prediction)):
        output.extend(prediction[i])
    prediction = output

    # Generate error data
    mse = mean_squared_error(data.tail(len(prediction)).values, prediction)
    rmse = mse ** 0.5
    mape = mean_absolute_percentage_error(data.tail(len(prediction)).reset_index(drop=True), pd.Series(prediction))
    return prediction, mse, rmse, mape




In [18]:
# if __name__ == '__main__':
    # Load historical data
    # CSV should have columns: ['date', 'open', 'high', 'low', 'close', 'volume']
data = pd.read_csv('SPY_bar_sample_10min.csv', index_col=0, header=0).tail(1500).reset_index(drop=False)
data['volume'] = data['volume'].astype('int32')
del data["average"]
del data["barcount"]

# Initialize moving averages from Ta-Lib, store functions in dictionary
talib_moving_averages = ['SMA', 'EMA', 'WMA', 'DEMA', 'KAMA', 'MIDPOINT', 'MIDPRICE', 'T3', 'TEMA', 'TRIMA']
functions = {}
for ma in talib_moving_averages:
    functions[ma] = abstract.Function(ma)
    
# Determine kurtosis "K" values for MA period 4-99
kurtosis_results = {'period': []}
for i in range(4, 100):
    kurtosis_results['period'].append(i)
    for ma in talib_moving_averages:
        
        # Run moving average, remove last 252 days (used later for test data set), trim MA result to last 60 days
        ma_output = functions[ma](data[:-252], i).tail(60)

        # Determine kurtosis "K" value
        k = kurtosis(ma_output, fisher=False)

        # add to dictionary
        if ma not in kurtosis_results.keys():
            kurtosis_results[ma] = []
        kurtosis_results[ma].append(k)

kurtosis_results = pd.DataFrame(kurtosis_results)


In [19]:
display(kurtosis_results)

,period,SMA,EMA,WMA,DEMA,KAMA,MIDPOINT,MIDPRICE,T3,TEMA,TRIMA
0,4,4.459327,4.150169,3.756510,3.745199,7.631069,4.421332,5.782130,4.707325,4.133767,4.633581
1,5,5.142722,4.919883,4.091829,3.536003,6.588201,4.478063,5.199357,6.041361,3.911231,5.210874
2,6,6.015571,5.738469,4.645159,3.392898,4.910843,4.565517,4.716487,7.446217,3.742465,5.848507
3,7,7.376823,6.399134,5.383712,3.291458,4.890520,5.594703,4.347899,8.419695,3.626023,6.651451
4,8,8.380819,6.816988,6.190828,3.221630,5.683777,6.049313,4.100213,8.750793,3.546664,7.527316
...,...,...,...,...,...,...,...,...,...,...,...
91,95,2.720072,2.008809,1.951291,2.571409,2.641678,4.038864,4.304235,1.777207,3.138310,2.430710
92,96,2.648108,2.004796,1.957686,2.559194,2.592129,4.038864,4.304235,1.780744,3.121144,2.470575
93,97,2.569965,2.000900,1.963239,2.547259,2.566300,4.038864,6.159115,1.784206,3.104627,2.507516
94,98,2.489100,1.997117,1.967910,2.535593,2.539490,4.038864,6.292633,1.787564,3.088725,2.542029


In [ ]:


# Determine period with K closest to 3 +/-5%
optimized_period = {}
for ma in talib_moving_averages:
    difference = np.abs(kurtosis_results[ma] - 3)
    df = pd.DataFrame({'difference': difference, 'period': kurtosis_results['period']})
    df = df.sort_values(by=['difference'], ascending=True).reset_index(drop=True)
    if df.at[0, 'difference'] < 3 * 0.05:
        optimized_period[ma] = int(df.at[0, 'period'])
    else:
        print(ma + ' is not viable, best K greater or less than 3 +/-5%')

print('\nOptimized periods:', optimized_period)

simulation = {}
for ma in optimized_period:
    # Split data into low volatility and high volatility time series
    low_vol = functions[ma](data, optimized_period[ma])
    high_vol = data['close'] - low_vol

    # Generate ARIMA and LSTM predictions
    print('\nWorking on ' + ma + ' predictions')
    try:
        low_vol_prediction, low_vol_mse, low_vol_rmse, low_vol_mape = get_arima(low_vol, 1000, 252)
    except:
        print('ARIMA error, skipping to next MA type')
        continue
    high_vol_prediction, high_vol_mse, high_vol_rmse, high_vol_mape = get_lstm(high_vol, 1000, 252)

    final_prediction = pd.Series(low_vol_prediction) + pd.Series(high_vol_prediction)
    mse = mean_squared_error(final_prediction.values, data['close'].tail(252).values)
    rmse = mse ** 0.5
    mape = mean_absolute_percentage_error(data['close'].tail(252).reset_index(drop=True), final_prediction)

    # Generate prediction accuracy
    actual = data['close'].tail(252).values
    result_1 = []
    result_2 = []
    for i in range(1, len(final_prediction)):
        # Compare prediction to previous close price
        if final_prediction[i] > actual[i-1] and actual[i] > actual[i-1]:
            result_1.append(1)
        elif final_prediction[i] < actual[i-1] and actual[i] < actual[i-1]:
            result_1.append(1)
        else:
            result_1.append(0)

        # Compare prediction to previous prediction
        if final_prediction[i] > final_prediction[i-1] and actual[i] > actual[i-1]:
            result_2.append(1)
        elif final_prediction[i] < final_prediction[i-1] and actual[i] < actual[i-1]:
            result_2.append(1)
        else:
            result_2.append(0)

    accuracy_1 = np.mean(result_1)
    accuracy_2 = np.mean(result_2)

    simulation[ma] = {'low_vol': {'prediction': low_vol_prediction, 'mse': low_vol_mse,
                                  'rmse': low_vol_rmse, 'mape': low_vol_mape},
                      'high_vol': {'prediction': high_vol_prediction, 'mse': high_vol_mse,
                                   'rmse': high_vol_rmse},
                      'final': {'prediction': final_prediction.values.tolist(), 'mse': mse,
                                'rmse': rmse, 'mape': mape},
                      'accuracy': {'prediction vs close': accuracy_1, 'prediction vs prediction': accuracy_2}}



Optimized periods: {'SMA': 80, 'EMA': 36, 'WMA': 46, 'DEMA': 71, 'KAMA': 83, 'MIDPOINT': 60, 'MIDPRICE': 18, 'T3': 23, 'TEMA': 25, 'TRIMA': 36}

Working on SMA predictions
Performing stepwise search to minimize aic
 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=-8122.755, Time=0.81 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=-8128.065, Time=0.44 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=-8126.709, Time=0.69 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=-8126.725, Time=0.73 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=-8129.782, Time=0.27 sec
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=-8125.130, Time=0.38 sec

Best model:  ARIMA(0,2,0)(0,0,0)[0]          
Total fit time: 3.323 seconds
ARIMA order: (0, 2, 0) 

working on 1 of 252 -- 0% complete
working on 2 of 252 -- 0% complete
working on 3 of 252 -- 1% complete
working on 4 of 252 -- 1% complete
working on 5 of 252 -- 1% complete
working on 6 of 252 -- 2% complete
working on 7 of 252 -- 2% complete
working on 8 of 252 -- 3% complete


/opt/conda/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


working on 18 of 252 -- 7% complete
working on 19 of 252 -- 7% complete
working on 20 of 252 -- 7% complete
working on 21 of 252 -- 8% complete
working on 22 of 252 -- 8% complete


/opt/conda/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


working on 23 of 252 -- 9% complete
working on 24 of 252 -- 9% complete
working on 25 of 252 -- 9% complete
working on 26 of 252 -- 10% complete
working on 27 of 252 -- 10% complete
working on 28 of 252 -- 11% complete
working on 29 of 252 -- 11% complete
working on 30 of 252 -- 11% complete
working on 31 of 252 -- 12% complete
working on 32 of 252 -- 12% complete
working on 33 of 252 -- 13% complete


...

working on 129 of 252 -- 51% complete
working on 130 of 252 -- 51% complete
working on 131 of 252 -- 51% complete
working on 132 of 252 -- 52% complete
working on 133 of 252 -- 52% complete
working on 134 of 252 -- 53% complete
working on 135 of 252 -- 53% complete
working on 136 of 252 -- 53% complete


/opt/conda/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


working on 137 of 252 -- 54% complete
working on 138 of 252 -- 54% complete


/opt/conda/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


working on 139 of 252 -- 55% complete
working on 140 of 252 -- 55% complete
working on 141 of 252 -- 55% complete
working on 142 of 252 -- 56% complete
working on 143 of 252 -- 56% complete
working on 144 of 252 -- 57% complete


In [ ]:
high_vol_prediction, high_vol_mse, high_vol_rmse, high_vol_mape = get_lstm(high_vol, 1000, 252)

final_prediction = pd.Series(low_vol_prediction) + pd.Series(high_vol_prediction)
mse = mean_squared_error(final_prediction.values, data['close'].tail(252).values)
rmse = mse ** 0.5
mape = mean_absolute_percentage_error(data['close'].tail(252).reset_index(drop=True), final_prediction)

# Generate prediction accuracy
actual = data['close'].tail(252).values
result_1 = []
result_2 = []
for i in range(1, len(final_prediction)):
    # Compare prediction to previous close price
    if final_prediction[i] > actual[i-1] and actual[i] > actual[i-1]:
        result_1.append(1)
    elif final_prediction[i] < actual[i-1] and actual[i] < actual[i-1]:
        result_1.append(1)
    else:
        result_1.append(0)

    # Compare prediction to previous prediction
    if final_prediction[i] > final_prediction[i-1] and actual[i] > actual[i-1]:
        result_2.append(1)
    elif final_prediction[i] < final_prediction[i-1] and actual[i] < actual[i-1]:
        result_2.append(1)
    else:
        result_2.append(0)

accuracy_1 = np.mean(result_1)
accuracy_2 = np.mean(result_2)

simulation[ma] = {'low_vol': {'prediction': low_vol_prediction, 'mse': low_vol_mse,
                              'rmse': low_vol_rmse, 'mape': low_vol_mape},
                  'high_vol': {'prediction': high_vol_prediction, 'mse': high_vol_mse,
                               'rmse': high_vol_rmse},
                  'final': {'prediction': final_prediction.values.tolist(), 'mse': mse,
                            'rmse': rmse, 'mape': mape},
                  'accuracy': {'prediction vs close': accuracy_1, 'prediction vs prediction': accuracy_2}}

# save simulation data here as checkpoint
with open('simulation_data.json', 'w') as fp:
    json.dump(simulation, fp)

for ma in simulation.keys():
print('\n' + ma)
print('Prediction vs Close:\t\t' + str(round(100*simulation[ma]['accuracy']['prediction vs close'], 2))
      + '% Accuracy')
print('Prediction vs Prediction:\t' + str(round(100*simulation[ma]['accuracy']['prediction vs prediction'], 2))
      + '% Accuracy')
print('MSE:\t', simulation[ma]['final']['mse'],
      '\nRMSE:\t', simulation[ma]['final']['rmse'],
      '\nMAPE:\t', simulation[ma]['final']['mape'])